In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch #for neural network 
import torch.nn as nn #neural netwoek funnc=torch.nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim #optimizer 
from torch.autograd import Function # for grdient calculcation
import pandas as pd #csv read 
import copy

from google.colab import drive
drive.mount('/content/drive') 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(device)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
cuda:0


In [ ]:
data = pd.read_csv('/content/drive/My Drive/World happinese/data.csv')
print(data)
data_xy = data.to_numpy() 


          GDP    Family    Health  ...  Corruption  Generosity  Happiness Score
0    1.396510  1.349510  0.941430  ...    0.419780    0.296780            7.587
1    1.302320  1.402230  0.947840  ...    0.141450    0.436300            7.561
2    1.325480  1.360580  0.874640  ...    0.483570    0.341390            7.527
3    1.459000  1.330950  0.885210  ...    0.365030    0.346990            7.522
4    1.326290  1.322610  0.905630  ...    0.329570    0.458110            7.427
..        ...       ...       ...  ...         ...         ...              ...
465  0.368746  0.945707  0.326425  ...    0.252756    0.455220            3.471
466  0.777153  0.396103  0.500533  ...    0.493664    0.151347            3.462
467  0.511136  1.041990  0.364509  ...    0.354256    0.066035            3.349
468  0.091623  0.629794  0.151611  ...    0.204435    0.084148            2.905
469  0.000000  0.000000  0.018773  ...    0.280876    0.056565            2.693

[470 rows x 7 columns]


In [ ]:
class Net(torch.nn.Module): 
  def __init__(self):
    super(Net, self).__init__() 
    self.fc1 = torch.nn.Linear(6, 200) #regression model 
    self.fc2 = torch.nn.Linear(200, 100)
    self.fc3 = torch.nn.Linear(100, 1)

  def forward(self, x): 
    x = F.relu(self.fc1(x)) 
    x = F.relu(self.fc2(x)) 
    x = self.fc3(x)
    return x

model = Net().to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-5, amsgrad=True) 
criterion = nn.MSELoss(reduction= 'sum') 

In [ ]:
for j in range(1000):
  np.random.shuffle(data_xy)
  train, test = data_xy[:329,:], data_xy[:141,:] #329 train ,141 test 
  # train
  train_x = train[:,:-1] 
  train_y = train[:,-1] 
  train_list = [] 
  for i in range(len(train_y)):
    train_list.append((train_x[i],train_y[i]))#
  # test
  test_x = test[:,:-1]
  test_y = test[:,-1]
  test_list = []
  for i in range(len(test_y)):
    test_list.append((test_x[i],test_y[i]))

  train_loader = torch.utils.data.DataLoader(train_list, batch_size=27) 
  test_loader = torch.utils.data.DataLoader(test_list, batch_size=141)
  previous_loss = 1000000 
  model.train()  
  for i in range(0,1000):
    for x, y in train_loader:
      x = Variable(x).to(device)
      y = Variable(y).to(device)
      y = y.reshape(len(y),1) # 
      predict = model(x.float()) 
      loss = criterion(predict, y)
      optimizer.zero_grad() 
      loss.backward()
      optimizer.step()
      if(loss.item()<previous_loss):
        previous_loss = loss.item()
      else:
        break
    # print('Epoch: ',i,' Loss: ',loss.item())

  test_x, test_y in test_loader
  model.eval() #prediction 
  acc1 = 0
  acc2 = 0 #
  predicted = model(torch.from_numpy(test_x).to(device).float()) 
  print('test_y ',end='')
  for i in test_y:
    print(i,end=' ')
  print()
  print('predict_y ',end='')
  for i in predicted:
    print(i.item(),end=' ')
  print()
  for i in range(len(test_y)):
    acc1 += abs(test_y[i]-predicted[i].item()) 
  print(j, ' : ',acc1/len(test_y)) 

torch.save({
            'model_state_dict': model.state_dict(),
            'model_optimizer_state_dict': optimizer.state_dict(),
            }, "/content/drive/My Drive/World happinese/model_V2.pt")

Output hidden; open in https://colab.research.google.com to view.